In [34]:
from bs4 import BeautifulSoup     
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import sys
import re
import os
import numpy as np

In [27]:
query_txt=input('공고명으로 검색할 키워드는 무엇입니까?: ')
startTime=input('조회 시작일자 입력(예:2019/01/01): ')
endTime=input('조회 종료일자 입력(예:2019/03/31): ')
f_name = input('파일로 저장할 폴더 이름을 쓰세요(예:c:\data\): ')

# query_txt='전기'
# startTime='2019/01/01'
# endTime='2019/03/31'
# f_name = '/Users/hwangseho/Desktop/SNSbigdata_sourcecode/final_res/Q1/'

In [28]:
path = '/opt/homebrew/Caskroom/chromedriver/112.0.5615.49/chromedriver'
driver = webdriver.Chrome(path)
driver.get("https://www.g2b.go.kr/index.jsp")
time.sleep(2)

/var/folders/g7/dh8v6rd91kz0fm7zl5kcjvq40000gn/T/ipykernel_42581/586772514.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [29]:
element=driver.find_element(By.ID, 'bidNm').click()
driver.find_element(By.XPATH, '//*[@id="bidNm"]').send_keys(query_txt)

element=driver.find_element(By.ID, 'fromBidDt')
ActionChains(driver).double_click(driver.find_element(By.ID, 'fromBidDt')).click().perform()
driver.find_element(By.XPATH, '//*[@id="fromBidDt"]').send_keys(startTime)

element=driver.find_element(By.ID, 'toBidDt')
ActionChains(driver).double_click(driver.find_element(By.ID, 'toBidDt')).click().perform()
driver.find_element(By.XPATH, '//*[@id="toBidDt"]').send_keys(endTime)

element=driver.find_element(By.CLASS_NAME, 'btn_dark').click()
time.sleep(1)

In [31]:
full_html = driver.page_source
soup = BeautifulSoup(full_html, 'html.parser')
# content_list = soup.find('table',attrs={'class':'table_list_tbidTbl table_list'})
content_list = soup.find_all('script')
content_str = str(content_list)

In [32]:
pattern  = r'https(.*?)"'
href_value = re.search(pattern, content_str).group().strip('"')
driver.get(href_value)
time.sleep(1)

In [35]:
cnt=1
while(1):
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')

    content=soup.find('table')
    content_list=content.find('tbody').find_all('td')

    if(content_list[0].get_text()=='검색된데이터가없습니다.'):
        break
    if(cnt==1):
        content_tr=content.find('thead').find_all('th')
        title=[i.get_text()foriincontent_tr]
        con_list=np.array(title)
    con=[]
    con.append(content_list[0].get_text())

    for i inrange(1,len(content_list)):
        if(i%10)==0:
            con_list=np.vstack((con_list,con))
            con=[]
        con.append(content_list[i].get_text())

    con_list=np.vstack((con_list,con))
    driver.execute_script('to_more(1)')
    cnt+=1

driver.quit()

In [36]:
os.chdir(f_name)
np.savetxt('Q1.csv',con_list,delimiter=",", fmt='%s')